In [2]:
import pandas as pd

In [117]:
csv = 'train_wkt_v4.csv'
annotations = pd.read_csv(csv)
is_building = u['ClassType']==1
building_only = annotations[is_building]
is_building_not_empty =  building_only['MultipolygonWKT']!='MULTIPOLYGON EMPTY'
buildings = building_only[is_building_not_empty]

In [118]:
buildings

,ImageId,ClassType,MultipolygonWKT
10,6120_2_2,1,MULTIPOLYGON (((0.000439 -0.009039999999999999...
20,6120_2_0,1,"MULTIPOLYGON (((0.005324 -0.002879, 0.005327 -..."
60,6100_1_3,1,"MULTIPOLYGON (((0.000748 -0.009042, 0.000745 -..."
80,6110_4_0,1,"MULTIPOLYGON (((0.007546 -0.005158, 0.007584 -..."
90,6140_3_1,1,MULTIPOLYGON (((0.0041385419488102 -0.00889310...
100,6110_1_2,1,MULTIPOLYGON (((5.99993550571e-05 -0.009031902...
110,6100_2_3,1,MULTIPOLYGON (((0.008963998153499299 -0.009041...
140,6140_1_2,1,MULTIPOLYGON (((0.000102 -0.009039999999999999...
150,6110_3_1,1,MULTIPOLYGON (((0.0040671780447186 -0.00889538...
200,6060_2_3,1,"MULTIPOLYGON (((0.006565 -0.008494, 0.006652 -..."
